In [2]:
# Import the dependencies
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# ML Intro
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import time

# GNN
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import os
import shutil
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.optim as optim
from random import shuffle
from torch_scatter import scatter_add
from torch.optim.lr_scheduler import ExponentialLR, MultiStepLR, ReduceLROnPlateau

# visualization
import networkx as nx
from torch_geometric.utils import to_networkx
import re
import cv2
from IPython.display import Video

### Displacement in a system of sheared grains

The particles in the system begin at rest, and as the system is sheared, the 'information' of the shearing displacement percolates from the moving walls up and down through the system.

To predict the displacement of our system, we need to understand the target and its complexities. Let's first create some visualizations of our data to study its characteristics. For times at which the information has completely penetrated the system, we might expect the particles to displace in an affine manner. (Note: this is because there are no pins in the configurations which we are studying here. If we were to introduce pins, we'd see some non-affine displacements.)

The figure below from Amin Danesh (Bucknell) illustrates the nature of this affine displacement as a function of y-coordinate in the simmulation.

![Affine displacements courtesy of Amin](../images/affine_displacement.png)

We'll use the LAMMPS dump files in the $\texttt{MD\_Data}$ directory to create our visualizations.

First we define functions for file IO and data accumulation:

In [7]:
def get_filename_for_timestep(timestep, folder='.'):
    """
    Returns the filename in `folder` corresponding to the timestep.
    Looks for files matching pattern 'confdump*MD<timestep>.data'.
    """
    pattern = re.compile(r'confdump.*MD' + str(timestep) + r'\.data')
    for filename in os.listdir(folder):
        if pattern.match(filename):
            return os.path.join(folder, filename)
    raise FileNotFoundError(f"No file found for timestep {timestep}")

In [8]:
def read_simulation_metadata(filename):
    with open(filename, 'r') as f:
        lines = [next(f) for _ in range(8)]
    Nall = int(lines[3].split()[0])
    Lx = float(lines[5].split()[1])
    Ly = float(lines[6].split()[1])
    return Nall, Lx, Ly

def load_particle_data(filename):
    header = np.loadtxt(filename, skiprows=8, max_rows=1, dtype=str)
    header[4:6] = ['dx', 'dy']  # Rename columns
    data = np.loadtxt(filename, skiprows=9, usecols=range(17), unpack=True)
    return header, data

def read_wall_particles(filename):
    with open(filename, 'r') as f:
        lines = [next(f) for _ in range(8)]
    Nwall = int(lines[3].split()[0])
    data = np.loadtxt(filename, skiprows=9, usecols=(0, 1), unpack=True)
    return Nwall, data

def extract_wall_indices(idarrayall, idarraywall):
    """Returns boolean mask or indices for wall particles."""
    return np.isin(idarrayall, idarraywall)

def data_func(tMD, t_0, folder = '../MD_Data', unwrapped_coords = True):
    """Processes simulation and wall data to extract displacement and other arrays."""
    file_wall = os.path.join(folder, "wallpartidtype")
    file_init = get_filename_for_timestep(t_0, folder)
    file_final = get_filename_for_timestep(tMD, folder)
    coords = "unwrapped" if unwrapped_coords else "wrapped"
    print(f"Loading data from final time: {file_final}")
    print(f"Loading data from init time: {file_init}")
    print(f"in {coords} coordinates!")

    Nall, Lx, Ly = read_simulation_metadata(file_final)

    Nall_init, Lx_init, Ly_init = read_simulation_metadata(file_init)
    
    if Nall != Nall_init:
        raise Exception('MD system configuration at final time step does not match initial time step.')

    # Read wall data
    Nwall, wall_data = read_wall_particles(file_wall)
    idarraywall, typearraywall = wall_data
    Nmid = Nall - Nwall

    # Load particle data
    header, data_final = load_particle_data(file_final)
    _, data_init = load_particle_data(file_init)

    # use unwrapped coords (fields 8,9) rather than periodic ones (fields 2,3)
    idarrayall = data_final[0]
    typearrayall = data_final[1]
    if unwrapped_coords:
        x_unwrapped_all, yarrayall = data_final[8], data_final[9]
    else:
        x_unwrapped_all, yarrayall = data_final[2], data_final[3]
    vxarrayall, vyarrayall = data_final[4], data_final[5]
    fxarrayall, fyarrayall = data_final[6], data_final[7]
    # skip columns 9-16 if unused

    if unwrapped_coords:
        x_unwrapped_all_init, yarrayall_init = data_init[8], data_init[9]
    else:
        x_unwrapped_all_init, yarrayall_init = data_init[2], data_init[3]
    vxarrayall_init, vyarrayall_init = data_init[4], data_init[5]

    dxarrayall = x_unwrapped_all - x_unwrapped_all_init
    dyarrayall = yarrayall - yarrayall_init

    # Get wall indices
    wall_mask = extract_wall_indices(idarrayall, idarraywall)

    # Extract mid particles (inverse mask)
    mid_mask = ~wall_mask
    idarraymid = idarrayall[mid_mask].astype(int)
    typearraymid = typearrayall[mid_mask].astype(int)
    xarraymid = x_unwrapped_all[mid_mask]
    yarraymid = yarrayall[mid_mask]
    xarraymid_init = x_unwrapped_all_init[mid_mask]
    yarraymid_init = yarrayall_init[mid_mask]
    dxarraymid = xarraymid - xarraymid_init
    dyarraymid = yarraymid - yarraymid_init
    vxarraymid = vxarrayall[mid_mask]
    vyarraymid = vyarrayall[mid_mask]
    fxarraymid = fxarrayall[mid_mask]
    fyarraymid = fyarrayall[mid_mask]
    # sigxyarraymid = sigxyarrayall[mid_mask]

    # Extract wall coordinates
    wall_indices = np.searchsorted(idarrayall, idarraywall)
    xarraywall = x_unwrapped_all[wall_indices]
    yarraywall = yarrayall[wall_indices]
    xarraywall_init = x_unwrapped_all_init[wall_indices]
    yarraywall_init = yarrayall_init[wall_indices]

    # Count mid region types
    Namid = np.sum(typearraymid == 1)
    Nbmid = np.sum(typearraymid == 2)
    Npinmid = np.sum(typearraymid == 3)

    return (header[2:8], idarrayall, typearrayall, dxarrayall, dyarrayall,
            vxarrayall, vyarrayall, xarraymid, yarraymid, xarraywall, yarraywall,
            dxarraymid, dyarraymid, typearraywall.astype(int), typearraymid.astype(int),
            Nmid, Nwall, Lx, Ly)

Now we can plot histograms as well as visualizations of the system at various time steps.

In [10]:
def plot_displacement_histograms(x_displacements, y_displacements, types, tMD, t_0):
    """Plots histograms for displacement data."""

    def _plot_and_save(x, y, net, label):
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        for ax, data, title, color in zip(
            axes,
            [x, y, net],
            ['X Displacement', 'Y Displacement', 'Net Displacement'],
            ['blue', 'green', 'red']
        ):
            counts, bins, patches = ax.hist(data, bins=50, color=color, alpha=0.7, edgecolor='black')
            patch_heights = counts / counts.sum()
            for patch, height in zip(patches, patch_heights):
                patch.set_height(height)
            ax.set_title(title)
            ax.set_xlabel("Displacement")
            ax.set_ylabel("Fraction of Total")
            ax.set_ylim(0, 1)
        fig.suptitle(f"Displacements at {tMD} timesteps from {t_0}", fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(f"../images/plots_fromcode/disp_hist_{label}_{tMD}.png")
        plt.close()

    net_displacements = np.hypot(x_displacements, y_displacements)

    _plot_and_save(x_displacements, y_displacements, net_displacements, 'ALL')
    for ttype, label in [(1.0, 'A'), (2.0, 'B')]:
        mask = types == ttype
        _plot_and_save(x_displacements[mask], y_displacements[mask], net_displacements[mask], label)

def displacement_heatmap(xmid_unwrapped, ymid_unwrapped, xwall_unwrapped, ywall_unwrapped, dxmid_unwrapped, dymid_unwrapped, xmid_wrapped, ymid_wrapped, xwall_wrapped, ywall_wrapped, dxmid_wrapped, dymid_wrapped, typewall, typemid, t, t_0, Nmid, Nwall, Lx, Ly):
    """
    Plots displacement heatmaps.
    NOTE: we differentiate between wrapped and unwrapped here
    because it is nice to visualize the positions of atoms in
    wrapped coordinates (ie, they won't leave the box area),
    but we must calculate the actual displacements with the unwrapped ones!
                         
    """
    Ra, Rb, Rpindraw = 1.0, 1.4, 0.4
    net_disp = np.hypot(dxmid_unwrapped, dymid_unwrapped)
    displacements = {'NET': net_disp, 'X': np.abs(dxmid_unwrapped), 'Y': np.abs(dymid_unwrapped)}

    for label, disp in displacements.items():
        disp_norm = (disp - disp.min()) / (disp.ptp() if disp.ptp() > 0 else 1)
        colormaps = {1: cm.Reds, 2: cm.Blues}  # ← colormap functions, not arrays

        fig, ax = plt.subplots()
        for i in range(Nmid):
            r = Ra if typemid[i] == 1 else Rb if typemid[i] == 2 else Rpindraw
            if typemid[i] in colormaps:
                color = colormaps[typemid[i]](disp_norm[i])
            else:
                color = cm.Greys(disp_norm[i])
            ax.add_patch(plt.Circle((xmid_wrapped[i], ymid_wrapped[i]), r, color=color))

        for i in range(Nwall):
            r = Ra if typewall[i] == 1 else Rb
            ax.add_patch(plt.Circle((xwall_wrapped[i], ywall_wrapped[i]), r, fc='yellow', ec='yellow'))

        ax.set_xlim(0, Lx)
        ax.set_ylim(0, Ly)
        ax.set_aspect('equal', adjustable='box')
        ax.set_title(f"Displacements at {t} timesteps from {t_0}_{label}")
        plt.savefig(f"../images/plots_fromcode/disp_MD_{t}_{label}.png")
        plt.close()

In [21]:
print("What data would you like to visualize?")
print()
print("Dump files contained in this file environment:")
print(".     - Start at MD0, goes in increments of 5000")
print(".       up to 200000.")
time.sleep(0.5)
print(".     - Then goes from 41100000 to")
print(".       41595000 in increments of 5000.")
time.sleep(0.5)
print(".     - Then goes in increments of 5000 from")
print(".       MD42000000 to 42500000.")
time.sleep(0.5)
print(".     - Also includes 2000, 10000000, 10100000,")
print(".       20000000, 30000000, 55900000.")
time.sleep(1.5)

print("Would you like to manually input time steps to visualize?")
time.sleep(0.5)
manual = int(input("Yes (1) or No (2): "))

if manual == 1:
    dt = int(input("select time interval between sampled files: "))
    t_0 = int(input("select inital time: "))
    t_f = int(input("select final time: "))
    num_files = int((t_f - t_0) / dt)
    tMD_arr = np.linspace(t_0, t_f, num=num_files, dtype=int)
elif manual ==2:
    print("Selecting default time steps to visualize:")
    print("0, 50000, 150000, 200000, 10000000, 20000000, 41100000")
    t_0 = 0
    f_f = 41100000
    tMD_arr = [0, 50000, 200000, 10000000, 20000000, 41100000]

os.makedirs(f"../images/plots_fromcode", exist_ok=True)
os.makedirs(f"../images/videos_fromcode", exist_ok=True)

What data would you like to visualize?

Dump files contained in this file environment:
.     - Start at MD0, goes in increments of 5000
.       up to 200000.
.     - Then goes from 41100000 to
.       41595000 in increments of 5000.
.     - Then goes in increments of 5000 from
.       MD42000000 to 42500000.
.     - Also includes 2000, 10000000, 10100000,
.       20000000, 30000000, 55900000.
Would you like to manually input time steps to visualize?


Yes (1) or No (2):  2


Selecting default time steps to visualize:
0, 50000, 150000, 200000, 10000000, 20000000, 41100000


In [32]:
data_directory = "../MD_Data/"
for t in tMD_arr:
    header, idarrayall, typearrayall, dxarrayall_unwrapped, dyarrayall_unwrapped, vxarrayall_unwrapped, vyarrayall_unwrapped, \
        xarraymid_unwrapped, yarraymid_unwrapped, xarraywall_unwrapped, yarraywall_unwrapped, dxarraymid_unwrapped, dyarraymid_unwrapped, \
        typearraywall, typearraymid, Nmid, Nwall, Lx, Ly = data_func(t, t_0, folder = data_directory, unwrapped_coords=True)
    header, idarrayall, typearrayall, dxarrayall_wrapped, dyarrayall_wrapped, vxarrayall_wrapped, vyarrayall_wrapped, \
        xarraymid_wrapped, yarraymid_wrapped, xarraywall_wrapped, yarraywall_wrapped, dxarraymid_wrapped, dyarraymid_wrapped, \
        typearraywall, typearraymid, Nmid, Nwall, Lx, Ly = data_func(t, t_0, folder = data_directory, unwrapped_coords=False)
    print("-----------------------------------------")

    plot_displacement_histograms(dxarraymid_unwrapped, dyarraymid_unwrapped, typearraymid, str(t), str(t_0))
    
    displacement_heatmap(xarraymid_unwrapped, yarraymid_unwrapped, xarraywall_unwrapped, yarraywall_unwrapped,
                         dxarraymid_unwrapped, dyarraymid_unwrapped, xarraymid_wrapped, yarraymid_wrapped, 
                         xarraywall_wrapped, yarraywall_wrapped, dxarraymid_wrapped, dyarraymid_wrapped, 
                         typearraywall, typearraymid, str(t), str(t_0), Nmid, Nwall, Lx, Ly)

Loading data from final time: ../MD_Data/confdumpearlyMD0.data
Loading data from init time: ../MD_Data/confdumpearlyMD0.data
in unwrapped coordinates!
Loading data from final time: ../MD_Data/confdumpearlyMD0.data
Loading data from init time: ../MD_Data/confdumpearlyMD0.data
in wrapped coordinates!
-----------------------------------------
Loading data from final time: ../MD_Data/confdumpearlyMD50000.data
Loading data from init time: ../MD_Data/confdumpearlyMD0.data
in unwrapped coordinates!
Loading data from final time: ../MD_Data/confdumpearlyMD50000.data
Loading data from init time: ../MD_Data/confdumpearlyMD0.data
in wrapped coordinates!
-----------------------------------------
Loading data from final time: ../MD_Data/confdumpearlyMD200000.data
Loading data from init time: ../MD_Data/confdumpearlyMD0.data
in unwrapped coordinates!
Loading data from final time: ../MD_Data/confdumpearlyMD200000.data
Loading data from init time: ../MD_Data/confdumpearlyMD0.data
in wrapped coordinates

Finally to facilitate a nice analysis of the time-dependence of our system's displacements, we can make videos!

In [33]:
def movie_from_plots(plot_prefix, hist=False, heatmap=False, folder_path = '../images/'):
    if hist:
        pattern_hist = rf'disp_hist_{plot_prefix}_(\d+)\.png'
        img_path = os.path.join(folder_path, 'plots_fromcode')
        files_hist = [f for f in os.listdir(img_path) if re.match(pattern_hist, f)]
        files_hist_sorted = sorted(files_hist, key=lambda f: int(re.search(pattern_hist, f).group(1)))
        if files_hist_sorted:
            first_img = cv2.imread(os.path.join(img_path, files_hist_sorted[0]))
            height, width, layers = first_img.shape
            save_path = os.path.join(folder_path, 'videos_fromcode', f'hist_{plot_prefix}_video.mp4')
            out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), 0.5, (width, height))
            for filename in files_hist_sorted:
                img = cv2.imread(os.path.join(img_path, filename))
                out.write(img)
            out.release()
            print(f"Saved video: {save_path}")
        else:
            print(f"No histogram files found for prefix {plot_prefix}")
    if heatmap:
        pattern_heatmap = rf'disp_MD_(\d+)_{plot_prefix}\.png'
        img_path = os.path.join(folder_path, 'plots_fromcode')
        files_heatmap = [f for f in os.listdir(img_path) if re.match(pattern_heatmap, f)]
        files_heatmap_sorted = sorted(files_heatmap, key=lambda f: int(re.search(pattern_heatmap, f).group(1)))
        if files_heatmap_sorted:
            first_img = cv2.imread(os.path.join(img_path, files_heatmap_sorted[0]))
            height, width, layers = first_img.shape
            save_path = os.path.join(folder_path, 'videos_fromcode', f'heatmap_{plot_prefix}_video.mp4')
            out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), 0.5, (width, height))
            for filename in files_heatmap_sorted:
                img = cv2.imread(os.path.join(img_path, filename))
                out.write(img)
            out.release()
            print(f"Saved video: {save_path}")
        else:
            print(f"No heatmap files found for prefix {plot_prefix}")

In [34]:
hist_strings = ['A', 'B', 'ALL']
heatmap_strings = ['X', 'Y', 'NET']

for hist in hist_strings:
    movie_from_plots(hist, hist=True)
for heatmap in heatmap_strings:
    movie_from_plots(heatmap, heatmap=True)

Saved video: ../images/videos_fromcode/hist_A_video.mp4
Saved video: ../images/videos_fromcode/hist_B_video.mp4
Saved video: ../images/videos_fromcode/hist_ALL_video.mp4
Saved video: ../images/videos_fromcode/heatmap_X_video.mp4
Saved video: ../images/videos_fromcode/heatmap_Y_video.mp4
Saved video: ../images/videos_fromcode/heatmap_NET_video.mp4


Navigate to $\texttt{heatmap\_X\_video.mp4}$ and enjoy the video!

You'll notice the displacement information percolate through the system, and at very long time steps, we see some weird behavior.

We are unsure if this behavior is warranted or due to oddities of the LAMMPS dump file we're using. We expect affine displacement at long time steps averaged across many simulation runs, so perhaps this behavior is warranted for a single simulation run.

### Changing the pre-processor

We'd now like our pre-processor to pass the displacement data to the GNN rather than the number of neighbors. This really only amounts to getting the $\texttt{dxarrayall}$ array from above for the relevant time interval and using it in place of the number of neighbors per particle in our pre-processor code.

So, rather than burden you with that same data processing code to get the x displacement array, I'll import the function we need:

In [38]:
from pre_processors.Pre_processing_displacement_Rowan import process_lammps_dump
from customDataset import CustDataset

In [39]:
# now let's call it to create our Snapshot file
# let's use MD 0 as the initial time, and MD 50 000 as our final time:
process_lammps_dump("../MD_Data/confdumpearlyMD0.data", "../MD_Data/wallpartidtype", "../MD_Data/confdumpearlyMD0.data", "../MD_Data/confdumpearlyMD50000.data", cutoff_distance=5.0, train_fraction=0.9)

Output written to Snapshot_1.graphdata


Note: this overwrites the Snapshot file we created for the neighbors task above!